In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta

In [2]:
def bulldog_page_job_offers() -> list:

    bulldog_list = list()

    url = "https://bulldogjob.pl/companies/jobs/s/city,Warszawa,Wrocław,Remote/skills,Python/experienceLevel,junior"
    flag = True
    try:
        page = requests.get(url)
    except requests.exceptions.ConnectionError as err:
        flag = False
    if flag:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="__next")
        job_elements = results.find_all(
            "div",
            class_="py-6 md:py-8 px-8 flex flex-wrap relative bg-white mb-2 rounded-lg shadow",
        )

        for job_element in job_elements:
            bulldog_dict = dict()
            job_title_element = job_element.find(
                "h3", class_="text-c28 font-medium mb-3 w-full md:hidden"
            )
            if (
                "Fullstack" not in job_title_element.text
                and "DevOps" not in job_title_element.text
                and "Test" not in job_title_element.text
            ):
                position_element = job_element.find(
                    "p", class_="tracking-05 uppercase md:my-4"
                )

                company_element = job_element.find(
                    "p",
                    class_="text-sm md:text-xxs md:text-center my-2 font-medium text-gray-300",
                )

                links = job_element.find(
                    "h3", class_="text-c28 font-medium mb-3 w-full md:hidden"
                )

                for link in links:
                    link_url = link["href"]
                pattern = re.compile(r'https?://([\w.\.\-]+)')
                website_name = pattern.match(link_url)
                page_job_element = requests.get(link_url)
                soup_page = BeautifulSoup(page_job_element.content, "html.parser")
                publication_date_element = soup_page.find(
                    "p", class_="text-gray-300 text-xs xl:text-sm mb-0.5"
                )
                days_after_publication = int(
                    re.findall(r"\b\d+\b", publication_date_element.text.strip())[0]
                )
                publication_date = (
                    datetime.today() - (timedelta(days=days_after_publication))
                ).strftime("%Y-%m-%d")

                bulldog_dict["publication_date"] = publication_date
                bulldog_dict["company"] = company_element.text.strip()
                bulldog_dict["title"] = job_title_element.text.strip()
                bulldog_dict["position"] = position_element.text.strip().capitalize()
                bulldog_dict["link_url"] = link_url
                bulldog_dict["website"] = website_name[0]
                bulldog_list.append(bulldog_dict)
    return bulldog_list


In [3]:
import pandas as pd
import numpy as np

bulldog_list = bulldog_page_job_offers()
df_raw = pd.DataFrame.from_records(bulldog_list)
df = df_raw.copy()
df['publication_date'] = pd.to_datetime(df['publication_date'], infer_datetime_format=True)
df.drop_duplicates(subset=['publication_date', 'company', 'title'], inplace=True, ignore_index=True)

In [4]:
df

,publication_date,company,title,position,link_url,website
0,2022-08-04,Accenture Polska,Junior Data Engineer with Python (Communicatio...,Junior,https://bulldogjob.pl/companies/jobs/120362-ju...,https://bulldogjob.pl
